# Start a async sentiment analysis
Input objects dataframe see `docs/schemas/object.md`  

Ouput an AsyncJobGroup that can be used to complete the sentiment analysis.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

from phoenix.common import artifacts, run_datetime
from phoenix.common import utils
from phoenix.tag.third_party_models.aws_async import start_sentiment
from phoenix.tag.third_party_models.aws_async import job_types

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parameters
# See phoenix/common/run_datetime.py expected format of parameter
RUN_DATETIME = None

# See phoenix/common/artifacts/registry_environment.py expected format of parameter
ARTIFACTS_ENVIRONMENT_KEY = "local"

# Filters for batch
YEAR_FILTER = 2021
# Without zero padding
MONTH_FILTER = 8
OBJECT_TYPE = "facebook_posts"

# DataAccessRole
AWS_DATA_ACCESS_ROLE = "arn:aws:iam::018241716617:role/AWSServiceComprehend"

In [ ]:
if RUN_DATETIME:
    run_dt = run_datetime.from_file_safe_str(RUN_DATETIME)
else:
    run_dt = run_datetime.create_run_datetime_now()
    
url_config = {
    "YEAR_FILTER": YEAR_FILTER,
    "MONTH_FILTER": MONTH_FILTER,
    "OBJECT_TYPE": OBJECT_TYPE,
}
art_url_reg = artifacts.registry.ArtifactURLRegistry(run_dt, ARTIFACTS_ENVIRONMENT_KEY, artifacts.registry_mappers.get_default_mappers())
TAGGING_RUNS_URL_PIPELINE_BASE = art_url_reg.get_url("tagging_runs-pipeline_base", url_config)
TAGGING_RUNS_URL_OBJECTS = art_url_reg.get_url("tagging_runs-objects", url_config)
TAGGING_RUNS_URL_ASYNC_JOB_GROUP = art_url_reg.get_url("tagging_runs-async_job_group", url_config)
TAGGING_RUNS_URL_COMPREHEND_BASE = art_url_reg.get_url("tagging_runs-comprehend_base", url_config)

In [ ]:
# Display params.
print(
TAGGING_RUNS_URL_PIPELINE_BASE,
TAGGING_RUNS_URL_OBJECTS,
TAGGING_RUNS_URL_ASYNC_JOB_GROUP,
TAGGING_RUNS_URL_COMPREHEND_BASE,
run_dt.dt,
AWS_DATA_ACCESS_ROLE,
sep='\n',
)

In [ ]:
objects = artifacts.dataframes.get(TAGGING_RUNS_URL_OBJECTS).dataframe

In [ ]:
objects.head()

In [ ]:
async_job_group = start_sentiment.start_sentiment_analysis_jobs(AWS_DATA_ACCESS_ROLE, TAGGING_RUNS_URL_COMPREHEND_BASE, objects)

In [ ]:
async_job_group

In [ ]:
_ = job_types.persist_json(TAGGING_RUNS_URL_ASYNC_JOB_GROUP, async_job_group)